Tranformar os meshs em uma descrição de grafo, começando por listar os vértices, normal, informação de textura e luminosidade.
Após isso, normalizar para um formato de arquivo já consolidade (npy)
Listar os vizinhos de cada vértice

In [ ]:
import re
import numpy
import torch
import trimesh
import networkx
import matplotlib.pyplot as plot

from skimage import color, graph
from disf import DISF_Superpixels
from skimage.segmentation import slic
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx

In [ ]:
def extract_attributes(line: str, attributes_pattern: re.Pattern) -> list:
    line = line.replace("  ", " ")

    if attributes_pattern.match(line):
        coord: list[str] = line.split(" ")
        if len(coord) < 4:
            coord.append(0.0)
        aList: list[float] = [float(coord[1]), float(coord[2]), float(coord[3])]
        return aList

In [ ]:
def extract_face(line: str, face_pattern: re.Pattern) -> list[list]:
    line = line.replace("  ", " ")
    indexes_attributes: list[list] = []

    if face_pattern.match(line):
        faces: list[str] = line.split(" ")
        for face in faces:
            indexes: list[str] = face.split("/")
            index_attribute: list[int] = []

            for index in indexes:
                try:
                    i = int(index)-1
                    index_attribute.append(i)
                except:
                    continue

            indexes_attributes.append([index for index in index_attribute])

    indexes_attributes.remove([])
    return indexes_attributes

In [ ]:
def to_graph(mesh_description_filename: str) -> Data:
    list_normals: list[list] = []
    list_vertices: list[list] = []
    list_textures: list[list] = []
    edges_index: list[list] = []
    faces_description: dict[str, list[list]] = []

    with open(file=mesh_description_filename, mode="r") as file:
        attributes_pattern: re.Pattern = re.compile(r'(v|vn|f|vt)(\s+-?\d+\.\d+){2,3}')
        face_pattern: re.Pattern = re.compile(r'f(\s+(\d+\/?){3}){3}')
        for line in file.readlines():
            aList: list = extract_attributes(line, attributes_pattern)

            if line.startswith("vn"):
                list_normals.append(aList)
            elif line.startswith("vt"):
                del aList[2]
                list_textures.append(aList)
            elif line.startswith("f"):
                faces = extract_face(line, face_pattern)
                edges_index.append([faces[0][0], faces[1][0]]) # vertices
                edges_index.append([faces[1][0], faces[2][0]])
                faces_description.append([faces[0][1], faces[1][1]]) # vertices textures
                faces_description.append([faces[1][1], faces[2][1]])
            elif line.startswith("v "):
                list_vertices.append(aList)
    normals = torch.tensor(list_normals, dtype=torch.long)
    vertices = torch.tensor(list_vertices, dtype=torch.long)
    textures = torch.tensor(list_textures, dtype=torch.float32)
    edges = torch.tensor(edges_index, dtype=torch.long)
    faces_attribtes = torch.tensor(faces_description, dtype=torch.long)

    data: Data = Data(x=vertices, edge_index=edges.t().contiguous(), edge_attr=faces_attribtes, norm=normals, pos=textures)


In [ ]:
"""
Data(x=[21661, 3], pos=[27355, 2], pos=[27355, 2], norm=[21661, 3])
x=[21661, 3]
[número de vértices, coordenadas do vértice]

edge_index=[2, 79998]
[dimensão da face, número de faces]

pos=[27355, 2]
[quantidade de texturas, coordenadas do UV mapping]

norm=[21661, 3]
[número de normal das vértices, coordenadas das normais de vértice]


(vertices, faces)
"""

data: Data = to_graph("data/0.obj")

In [ ]:
mesh = trimesh.load(file_obj="data/0.obj", file_type="obj", force="mesh")
vertices = torch.tensor(mesh.vertices, dtype=torch.float32)
faces = torch.tensor(mesh.faces, dtype=torch.long).t().contiguous()

In [ ]:
# data = Data(pos=vertices, face=faces)
# positions = data.pos.numpy()
# segments = slic(positions, n_segments=100, compactness=10, sigma=1, channel_axis=None)

# rag_graph = networkx.Graph()
# rag_graph.add_nodes_from(segments)
# for face in faces.numpy():
#     edge = (segments[face[0]], segments[face[1]])
#     rag_graph.add_edge(*edge)


# rag = networkx.rag_mean_color(color.rgb2gray(positions), segments)

In [ ]:
positions = mesh.edges
display(mesh)

# segments = slic(positions, n_segments=100, compactness=10, sigma=1, channel_axis=None)
# segments = DISF_Superpixels(positions.ravel(), 10_000, 100)

# Create Region Adjacency Graph (RAG) from the segments
# rag = graph.rag_mean_color(positions, segments, connectivity=10)

In [ ]:
# display(nx_graph)
# display(rag)
g = rag.copy()